In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as f
from torch import Tensor
from torch.utils.data import DataLoader,Dataset
 
import datetime
import time

import pandas as pd
import numpy as np
import pickle
import os
os.environ['CUDA_VISIBLE_DEVICES']='1'
import warnings
warnings.filterwarnings('ignore')

In [2]:
def exe_time(func):
    '''
    装饰器，返回原函数返回的，并打印出原函数前后的时间
    '''
    def new_func(*arg,**kwargs):
        name=func.__name__
        back=func(*args,**kwargs)
        start=datetime.datetime.now()
        end=datetime.datetime.now()
        total=(end-start).total_seconds()
        print('--{%s} start:@ %ss' % (name,start))
        print('--{%s} end:@ %ss'%(name,end))
        print('--{%s} total:@ %.3fs=%.3fh'%(name,total,total/3600.0))
        return back
    return new_func


In [3]:
##1定义MlP层
class MLP(nn.Module):
    def __init__(self,mlp_units,acti=torch.relu,size_x=216,rate=0.1):
        super(MLP,self).__init__()
        self.dropout=nn.Dropout(rate)
        
        self.layernorm1=nn.LayerNorm(mlp_units[0],eps=1e-6)
        self.layernorm2=nn.LayerNorm(mlp_units[1],eps=1e-6)

        self.dense1=nn.Linear(size_x,mlp_units[0])
        self.dense2=nn.Linear(mlp_units[0],mlp_units[1])
        self.dense_score=nn.Linear(mlp_units[1],mlp_units[2])
        self.acti1=acti
        self.acti2=torch.sigmoid
    def forward(self,x):
        x=self.dropout(x)
        x=self.acti1(self.dense1(x))
        x=self.layernorm1(x)
        
        x=self.dropout(x)
        x=self.acti2(self.dense2(x))
        x=self.layernorm2(x)

        y=self.dense_score(x)
        y=self.acti2(y)
        y=torch.squeeze(y,dim=-1)

        return y
    def fun_obtain_params(self):
        return [(name,param.shape) for name,param in self.named_parameters()]

In [4]:
#1.2测试MLP层
# '''[256,128,1]是对embed维度的操作'''
# model=MLP([256,128,1],acti=torch.relu,size_x=16,rate=0.1)
# model.train()
# a=time.time()
# for _ in range(1000):
#     user_emb=torch.tensor(np.random.uniform(size=(10,8)),
#                           dtype=torch.float32)
#     item_emb=torch.tensor(np.random.uniform(size=(10,8)),
#                           dtype=torch.float32)
#     embs=torch.cat((user_emb,item_emb),dim=1)
#     preference=model(embs)
# print(preference.shape,preference)
# print('time cost',time.time()-a)
# print('trainable_parameters:',model.fun_obtain_params())
# print('l2 loss:',sum(p.pow(2).sum() for p in model.parameters()))
# print('***mlp check:done')
# print()

In [5]:
#定义注意力层
class BahdanauAttentionQK_softmax(nn.Module):
    def __init__(self,size_x,d_model,rate=0.0):
        super(BahdanauAttentionQK_softmax,self).__init__()

        self.wq=nn.Linear(size_x,d_model)
        self.wk=nn.Linear(d_model,d_model)
        self.v=nn.Linear(d_model,1)
        self.dropout=nn.Dropout(rate)
    def forward(self,q,k,mask=None):
        '''
        q:(bs,seqlen,dim=152)
        k:(bs,seqlen,7,dim=64)
        '''
        q=self.dropout(q)
        k=self.dropout(k)

        query_with_time_axis=q.unsqueeze(2)
        tiled_tensor=query_with_time_axis.repeat(1,1,7,1)#(bs,seq_len,7,152)
        score=self.v(torch.tanh(self.wq(query_with_time_axis)
                                +self.wk(k)))#(bs,seq_len,7,64) (bs,sl,7,64)
        att_weights=torch.softmax(score,dim=-2)

        att_out=att_weights*k
        att_out=torch.sum(att_out,dim=-2)
        
        return att_out,att_weights#att_out注意力最后结果，att_weights注意力分数,(bs,sl,64),(bs,sl,7,1)

In [6]:
# attention_layer=BahdanauAttentionQK_softmax(8,10)
# attention_layer.train()
# #sample_hidden是q,（batch_size,seq_len);sample_output是k/v,(batch_size,k个数,seq_len)
# sample_hidden=torch.tensor(np.random.random(size=(4,8)),dtype=torch.float32)
# sample_output=torch.tensor(np.random.random(size=(4,5,8)),dtype=torch.float32)
# att_result,att_weights=attention_layer(sample_hidden,sample_output)
# print('att result shape:(batch_size,units){}'.format(att_result.shape))
# print('att weights shape:(batch_size,sequence_length,1){}'
#       .format(att_weights.shape))

In [7]:
class Preference_Model(nn.Module):
    def __init__(self,size_user,size_poi,size_cat_id,size_day,size_hour,size_hs5,
                 size_embed_user,size_embed_poi,size_embed_cat,size_embed_day,size_embed_hour,size_embed_hs5,
                 size_mlps,d_model,rate_mlp=0,rate_drop_preference=0.0):
        '''
        size_mlps:mlp层中的每层单元个数[256,128,1]
        size_dense(d_model):256,模型中隐藏单元数量
        '''
        super(Preference_Model,self).__init__()

        self.acti=torch.relu

        self.embed_user=nn.Embedding(size_user,size_embed_user)
        self.embed_poi=nn.Embedding(size_poi,size_embed_poi)
        self.embed_cat_id=nn.Embedding(size_cat_id,size_embed_cat)
        # self.embed_1st=nn.Embedding(size_1st,size_embed_1st)
        self.embed_day=nn.Embedding(size_day,size_embed_day)
        self.embed_hour=nn.Embedding(size_hour,size_embed_hour)
        self.embed_hs5=nn.Embedding(size_hs5,size_embed_hs5)
        
        data_size=size_embed_poi+size_embed_cat+size_embed_day+size_embed_hour+size_embed_hs5
        self.dense=nn.Linear(data_size,d_model)
        self.model_att_qk_sft=BahdanauAttentionQK_softmax(data_size,d_model,rate_drop_preference)
            # def __init__(self,mlp_units,acti=torch.relu,size_x=216,rate=0.1):
        self.model_mlp=MLP(size_mlps)

    def long_interest_att(self,embs_poi,embs_ctxt,sub_days_masks):
        '''
        sub_days_masks:(batch_size,seq_len,7)
        '''
        def divide_no_nan(x,y,nan_value=0.0):
            mask=y==0
            y=torch.where(mask,torch.ones_like(y),y)
            result=x/y
            result=torch.where(mask,torch.tensor(nan_value),result)
            return result
            
        sub_days_masks=sub_days_masks.float()
        cum_idxs=torch.cumsum(sub_days_masks,dim=1)
        cum_idxs_mask=(cum_idxs==0.).float()
        cum_idxs_mask=cum_idxs_mask.unsqueeze(-1)

        input_embs=torch.cat([embs_poi,embs_ctxt],dim=2)#152
        input_embs_expand=input_embs.unsqueeze(2)
        input_subs_idxs=sub_days_masks.unsqueeze(3)
        input_embs_expand_subs=input_embs_expand*input_subs_idxs

        input_subs_idxs_cum=torch.cumsum(input_subs_idxs,dim=1)
        input_embs_expand_subs_cum=torch.cumsum(input_embs_expand_subs,dim=1)
        input_embs_expand_subs_cum_avg=divide_no_nan(input_embs_expand_subs_cum,input_subs_idxs_cum)

        cum_subs_avg=input_embs_expand_subs_cum_avg#(b_s,seq_len,7,152)
        cum_subs_msk=cum_idxs_mask#(bs,seq_len,7,1)
        return cum_subs_avg,cum_subs_msk
        
    def forward(self,users,padded_pois,padded_cats_id,
                lens,padded_days,padded_hours,padded_hs5s,padded_sub_days_masks):#,padded_subs_1st_masks):
        embedded_users=self.embed_user(users)
        embedded_pois=self.embed_poi(padded_pois)
        embedded_cats_id=self.embed_cat_id(padded_cats_id)
        # embedded_1st=self.embed_1st(padded_1st_cats_id)
        embedded_days=self.embed_day(padded_days)
        embedded_hour=self.embed_hour(padded_hours)
        embedded_hs5s=self.embed_hs5(padded_hs5s)

        embs_poi=torch.cat((embedded_pois,embedded_cats_id),axis=-1)#64+32=96,(bs,seq_len,dim)
        embs_ctxt=torch.cat((embedded_days,embedded_hour,embedded_hs5s),axis=-1)#8+16+32=56
        
        #平均池化        
        cum_subs_avg,cum_subs_msk=self.long_interest_att(embs_poi,embs_ctxt,padded_sub_days_masks)
        #dense
        cum_subs_avg=self.dense(cum_subs_avg)#(bs,seq_l,7,dim)
        #att
        ipt_q=torch.cat((embs_poi,embs_ctxt),dim=2)#正确的数据做q(bs,seq_len,dim=152)
        k=cum_subs_avg#(bs,seq_len,7,dim=64)
        
        att_out,attw=self.model_att_qk_sft(ipt_q,k)

        concat=torch.cat((att_out,ipt_q),dim=-1)#(bs,sq,64+152=216)
        mlp_out=self.model_mlp(concat)
        return (mlp_out)



In [8]:
#1加载数据
#1.1加载Preference_Model初始化数据
size_user=1084#长度1078，最大1083,又因为实际是从min=1,max=1083,编码min=0,max=1083,编码长度为1084
size_poi=3906
size_cat_id=285
size_1st=9#用不到
size_day=8
size_hour=25
size_hs5=95

size_embed_user=64
size_embed_poi=64
size_embed_cat=32
size_embed_1st=8
size_embed_day=8
size_embed_hour=16
size_embed_hs5=32
size_mlps=[256,128,1]
d_model=64#即hidden_size

batch_size=10
epoch_size=20
lr=0.0001

In [9]:
#1.2测试初始化model
model=Preference_Model(size_user,size_poi,size_cat_id,size_day,size_hour,size_hs5,
                       size_embed_user,size_embed_poi,size_embed_cat,size_embed_day,size_embed_hour,size_embed_hs5,
                       size_mlps,d_model)

In [10]:
#2加载forward用的实际数据，一会放入dataloader中
#2.1加载路径
dir_users='/home/jovyan/datasets/tsmc_nyc_3_groupby_user_chronological/user_id.pkl'
#用到所有数据，所以要计算所有数据的实际长度，最大长度，以及填充它们
dir_padded_pois='/home/jovyan/datasets/tsmc_nyc_8_all_tgt_padding/padded_reindex_poi.pkl'
dir_padded_cats_id='/home/jovyan/datasets/tsmc_nyc_8_all_tgt_padding/padded_reindex_cat_id.pkl'
dir_lens='/home/jovyan/datasets/tsmc_nyc_8_all_tgt_padding/lens.pkl'
dir_padded_days='/home/jovyan/datasets/tsmc_nyc_8_all_tgt_padding/padded_days.pkl'
dir_padded_hours='/home/jovyan/datasets/tsmc_nyc_8_all_tgt_padding/padded_hours.pkl'
dir_padded_hs5s='/home/jovyan/datasets/tsmc_nyc_8_all_tgt_padding/padded_hs5.pkl'
#dir_padded_sub_days_masks用于做平均池化
dir_padded_sub_days_masks='/home/jovyan/datasets/tsmc_nyc_8_all_tgt_padding/padded_subs_days.pkl'
dir_padded_tgt='/home/jovyan/datasets/tsmc_nyc_8_all_tgt_padding/padded_tgt_reindex_poi.pkl'

#2.2加载实际传入forward的数据
# def forward(users,padded_pois,padded_cats_id,
#                 lens,padded_days,padded_hours,padded_hs5s,padded_sub_days_masks):
with open(dir_users,'rb') as f:
    users=pickle.load(f)
with open(dir_padded_pois,'rb') as f:
    padded_pois=pickle.load(f)
with open(dir_padded_cats_id,'rb') as f:
    padded_cats_id=pickle.load(f)
with open(dir_lens,'rb') as f:
    lens=pickle.load(f)
with open(dir_padded_days,'rb') as f:
    padded_days=pickle.load(f)
with open(dir_padded_hours,'rb') as f:
    padded_hours=pickle.load(f)
with open(dir_padded_hs5s,'rb') as f:
    padded_hs5s=pickle.load(f)
with open(dir_padded_sub_days_masks,'rb') as f:
    padded_sub_days_masks=pickle.load(f)
with open(dir_padded_tgt,'rb') as f:
    padded_tgt=pickle.load(f)

In [11]:
#3写dataloader
def Process_preference_model(users,padded_pois,padded_cats_id,padded_days,padded_hours,padded_hs5s,
                             padded_sub_days_masks,
                            tgt,batch_size):
    class PreDataset(Dataset):
        def __init__(self,users,padded_pois,padded_cats_id,padded_days,padded_hours,padded_hs5s,
                     padded_sub_days_masks,tgt):
            self.users=users
            self.padded_pois=padded_pois
            self.padded_cats_id=padded_cats_id
            self.padded_days=padded_days
            self.padded_hours=padded_hours
            self.padded_hs5s=padded_hs5s
            self.padded_sub_days_masks=padded_sub_days_masks
            self.tgt=tgt
        def __len__(self):
            return len(self.users)
        def __getitem__(self,index):
            user=self.users[index]
            poi=self.padded_pois[index]
            cats_id=self.padded_cats_id[index]
            days=self.padded_days[index]
            hours=self.padded_hours[index]
            hs5s=self.padded_hs5s[index]
            sub_days_masks=self.padded_sub_days_masks[index]
            tgt_item=self.tgt[index]
            return user,poi,cats_id,days,hours,hs5s,sub_days_masks,tgt_item
    #创建数据集实例
    all_dataset=PreDataset(users,padded_pois,padded_cats_id,padded_days,padded_hours,padded_hs5s,padded_sub_days_masks,tgt)
    dataloader=DataLoader(all_dataset,batch_size,shuffle=True)
    return dataloader

In [12]:
#3.1验证dataloader
# def Process_preference_model(users,padded_pois,padded_cats_id,padded_days,padded_hours,padded_hs5s,
#                              padded_sub_days_masks,
#                             tgt,batch_size):
pre_all_data=Process_preference_model(users,padded_pois,padded_cats_id,padded_days,padded_hours,padded_hs5s,
                                      padded_sub_days_masks,padded_tgt,batch_size)
# for batch_users,batch_poi,batch_cats,batch_days,batch_hours,batch_hs5s,batch_sub_days_masks,batch_tgt in pre_all_data:
#     print(f'batch data:{batch_users},{batch_poi},{batch_cats},{batch_days},{batch_hours},{batch_hs5s},{batch_sub_days_masks},{batch_tgt}')

In [13]:
#2.3定义评价矩阵
def Top_k_precision(indices, batch_y, k):
    '''
    indices:一个batch排序之后的下标，(batch_size,待预测长度),size_cat/size_poi，tensor
    batch_y:(batch_size)
    '''
    precision = 0
    for i in range(len(batch_y)):
        if batch_y[i] in indices[:k]:
            precision += 1
    return precision / indices.size(0)

In [14]:
loss_function=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr)

In [15]:
#4将数据传入模型
for epoch in range(epoch_size):
    model.train()
    epoch_loss=0.0
    top_k_1=0
    top_k_5=0
    top_k_10=0
    top_k_20=0
    data_len=len(pre_all_data)
    for batch_step,(batch_users,batch_poi,batch_cats,batch_days,batch_hours,
                    batch_hs5s,batch_sub_days_masks,batch_tgt) in enumerate(pre_all_data):
        model.zero_grad()
        batch_users=batch_users
        batch_poi=batch_poi
        batch_cats=batch_cats
        batch_days=batch_days
        batch_hs5s=batch_hs5s
        batch_sub_days_masks=batch_sub_days_masks
        batch_tgt=batch_tgt
        out=model(batch_users,batch_poi,batch_cats,lens,batch_days,batch_hours,batch_hs5s,batch_sub_days_masks)
        loss=0.0
        loss+=loss_function(out,torch.tensor(batch_tgt,dtype=torch.float))
        loss.backward()
        optimizer.step()
        epoch_loss+=float(loss)
        tgt_evaluation=[sliced_tensor[-1] for sliced_tensor in batch_tgt]
        top_k_1+=Top_k_precision(out,tgt_evaluation,1)
        # top_k_5+=Top_k_precision(indices,tgt_evaluation,5)
        # top_k_10+=Top_k_precision(indices,tgt_evaluation,10)
        # top_k_20+=Top_k_precision(indices,tgt_evaluation,20)
    print('epoch:[{}/{}]\t'.format(epoch,epoch_size),
          'loss:{:.4f}\t'.format(epoch_loss),
        'top@1:{:4f}\t'.format(top_k_1/data_len)
        # 'top@5:{:4f}\t'.format(top_k_5/data_len),
        # 'top@10:{:4f}\t'.format(top_k_10/data_len),
        # 'top@20:{:4f}\t'.format(top_k_20/data_len)
    )


epoch:[0/20]	 loss:79665906.5625	 top@1:0.000000	
epoch:[1/20]	 loss:77521022.5000	 top@1:0.000000	
epoch:[2/20]	 loss:77426421.0000	 top@1:0.000000	
epoch:[3/20]	 loss:77331643.1250	 top@1:0.000000	
epoch:[4/20]	 loss:77365076.0625	 top@1:0.000000	
epoch:[5/20]	 loss:77388602.8125	 top@1:0.000000	
epoch:[6/20]	 loss:77345886.8438	 top@1:0.000000	
epoch:[7/20]	 loss:77337461.9375	 top@1:0.000000	
epoch:[8/20]	 loss:77377782.4062	 top@1:0.000000	
epoch:[9/20]	 loss:77393418.4375	 top@1:0.000000	
epoch:[10/20]	 loss:77321196.1250	 top@1:0.000000	
epoch:[11/20]	 loss:77333638.3125	 top@1:0.000000	
epoch:[12/20]	 loss:77347313.0000	 top@1:0.000000	
epoch:[13/20]	 loss:77323546.6250	 top@1:0.000000	
epoch:[14/20]	 loss:77334877.9688	 top@1:0.000000	
epoch:[15/20]	 loss:77307532.5625	 top@1:0.000000	
epoch:[16/20]	 loss:77332785.0000	 top@1:0.000000	
epoch:[17/20]	 loss:77290445.2188	 top@1:0.000000	
epoch:[18/20]	 loss:77312008.7500	 top@1:0.000000	
epoch:[19/20]	 loss:77306858.2500	 top@1: